<a href="https://colab.research.google.com/github/Chocoding1/Machine_Learning_Deep_Learning/blob/main/%5BDeep%20Learning%5D%20GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN, 오토인코더

GAN : 생성적 적대 신경망(Generative Adversarial Networks)
- 딥러닝의 원리를 활용해 가상 이미지를 생성하는 알고리즘
- 진짜같은 가짜를 만들기 위해 GAN 알고리즘 내부에서 '적대적' 경합 진행
- 생성자(Generator) : 가짜 만드는 파트
- 판별자(Discriminator) : 진위를 가려내는 파트

여기서는 페이스북의 DCGAN(Deep Convolutional GAN)을 사용할 예정
- DCGAN : 컨볼루션 신경망(CNN)을 GAN에 적용한 알고리즘

## 생성자(Generator)

- 처음에는 랜덤한 픽셀 값으로 채워진 가짜 이미지로 시작해서 판별자의 판별 결과에 따라 지속적으로 업데이트하며 점차 원하는 이미지 생성
- DCGAN에서 사용되는 CNN은 앞서 나온 것과 달리 optimizer(최적화)나 컴파일 과정 필요 x -> 판별과 학습이 생성자에서 일어나는 것이 아니기 때문
- 풀링(pooling) 과정 없고, 패딩(padding) 과정만 포함 -> 입력 크기와 출력 크기를 동일하게 맞추기 위해(이미지가 컨볼루션 층을 지나면 크기가 줄어들기 때문)
- 패딩을 통해 이미지 크기를 조절해야 하는 정확한 이유는 판별자가 비교할 '진짜'와 똑같은 크기가 되어야 하기 때문(padding='same' : 입력과 출력의 크기가 다를 경우 자동으로 크기 확장(0으로 채움))
- 배치 정규화(Batch Normalization) : 입력 데이터의 평균이 0, 분산이 1이 되도록 재배치하는 것으로, 다음 층으로 입력될 값을 일정하게 재배치 하는 역할 -> 층의 개수가 늘어나도 안정적인 학습 진행 가능
- 생성자의 활성화 함수는 relu 함수, 판별자로 넘겨주기 직전에는 tanh 함수 사용(tanh 함수 : 출력값을 -1~1 사이로 조정)
- 판별자에 입력될 MNIST 손글씨 픽셀 범위도 -1~1로 맞추면 판별 조건이 모두 갖춰진다.

In [ ]:
# Generator 코드
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=100, activation='LeakyReLU(0.2)')) # 1
generator.add(BatchNormalization()) # 2
generator.add(Reshape(7, 7, 128)) # 3
generator.add(UpSampling2D()) # 4
generator.add(Conv2D(64, kernel_size=5, padding='same')) # 5
generator.add(BatchNormalization()) # 6
generator.add(Activation(LeakyReLU(0.2))) # 7
generator.add(UpSampling2D()) # 8
generator.add(Conv2D(1, kernel_size=5, padding='same', activation='tanh')) # 9

1. 128 : 임의로 정한 노드 수<br>
input_dim=100 : 임의로 100차원 크기의 랜덤 벡터 입력<br>
7 * 7 : 모델 구조를 보면 UpSampling2D()가 있는데, 이는 가로세로 크기를 두 배씩 늘려준다. MNIST 이미지 크기는 28x28, 즉 7x7 이미지를 총 2번 UpSampling2D()하여 28x28로 만들어주는 것(이렇게 작은 크기의 이미지를 점점 늘려 가면서 컨볼루션 층을 지나치게 하는 것이 DCGAN의 특징)<br>
keras는 UpSampling과 Conv2D를 합쳐놓은 Conv2DTranspose() 함수도 제공
2. 데이터의 배치를 정규 분포로 만든다.
3. convolution layer가 받아들일 수 있는 형태로 바꿔주는 코드(Conv2D() 함수의 input_shape 부분에 들어갈 형태로 지정)
4. 이미지 가로세로 길이 2배 증가
5. 컨볼루션 과정